# Detecting Retina Damage From Optical Coherence Tomography (OCT) Images, using Transfer Learning on VGG16 CNN Model

In [2]:
#!pip install keract

## Imports

In [3]:
import os
from glob import glob
import pandas as pd
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sn
from skimage.transform import resize
from skimage.color import gray2rgb
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import SVG
import keract
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications, optimizers
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.utils import to_categorical, model_to_dot, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau

## Dataset

In [4]:
data_dir = "../data/OCT2017/"
train_data_dir= '../data/OCT2017/train/'
val_data_dir= '../data/OCT2017/val/'
test_data_dir= '../data/OCT2017/test/'

In [5]:
img_width, img_height = 150, 150 
channels = 3
batch_size = 32

In [6]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

train_dataset_path = os.listdir(train_data_dir)
print("Types of classes found: ", len(train_dataset_path))

cnv_images = len(glob(train_data_dir + 'CNV/*.jpeg'))
dme_images = len(glob(train_data_dir + 'DME/*.jpeg'))
drusen_images = len(glob(train_data_dir + 'DRUSEN/*.jpeg'))
normal_images = len(glob(train_data_dir + 'NORMAL/*.jpeg'))

data= {'CNV': cnv_images, 'DME': dme_images, 'DRUSEN': drusen_images, 'NORMAL': normal_images}

labels = list(data.keys()) 
count = list(data.values())

print('cnv ', cnv_images, '\ndme ', dme_images,'\ndrusen ', drusen_images,'\nnormal ', normal_images)
print('total :', cnv_images+dme_images+drusen_images+normal_images)

Types of classes found:  4
cnv  37205 
dme  11348 
drusen  8616 
normal  26315
total : 83484


In [7]:
retinas = []

for item in train_dataset_path:
 # Get all the file names
 all_retinas = os.listdir(train_data_dir  +item)
 #print(all_shoes)

 # Add them to the list
 for retina in all_retinas:
    retinas.append((item, str(train_data_dir +item) + '/' + retina))

In [8]:
# Build a dataframe        
retinas_df = pd.DataFrame(data=retinas, columns=['classe', 'image'])
print(retinas_df.head())
#print(rooms_df.tail())

  classe                                           image
0    CNV    ../data/OCT2017/train/CNV/CNV-1016042-1.jpeg
1    CNV   ../data/OCT2017/train/CNV/CNV-1016042-10.jpeg
2    CNV  ../data/OCT2017/train/CNV/CNV-1016042-100.jpeg
3    CNV  ../data/OCT2017/train/CNV/CNV-1016042-101.jpeg
4    CNV  ../data/OCT2017/train/CNV/CNV-1016042-102.jpeg


In [9]:
# Let's check how many samples for each category are present
print("Total number of retinas in the train dataset: ", len(retinas_df))

retinas_count = retinas_df['classe'].value_counts()

print("retinas in each category: ")
print(retinas_count)

Total number of retinas in the train dataset:  83484
retinas in each category: 
CNV       37205
NORMAL    26315
DME       11348
DRUSEN     8616
Name: classe, dtype: int64


In [10]:
os.listdir(train_data_dir)


['CNV', 'DME', 'DRUSEN', 'NORMAL']

In [29]:
import cv2

im_size = 150

images = []
labels = []

for i in os.listdir(train_data_dir):
    print(i)
    data_path = train_data_dir + str(i)  
    filenames = [i for i in os.listdir(data_path)]
    
    for f in range(0,1000):
        img = cv2.imread(data_path + '/' + filenames[f])
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

CNV
DME
DRUSEN
NORMAL


In [16]:
images = np.array(images)
images = images.astype('float32') / 255.0
images.shape

(32000, 150, 150, 3)

In [17]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y = LabelEncoder().fit_transform(labels)

In [18]:
y=y.reshape(-1,1)
onehotencoder = OneHotEncoder(categories='auto')  #Converted  scalar output into vector output where the correct class will be 1 and other will be 0
Y= onehotencoder.fit_transform(y)
Y.shape  #(40, 2)

(32000, 4)

In [19]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(30400, 150, 150, 3)
(30400, 4)
(1600, 150, 150, 3)
(1600, 4)


## Model

### Transfert learning

* VGG16 CNN architecture is used for classification.
* Pretrained on the 'ImageNet' dataset.

In [20]:
# instanciation d'un model VGG16 avec pré-entrainement imagenet
vgg16 = VGG16(include_top= False, input_shape= (img_width, img_height, channels), weights= 'imagenet')
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [21]:
# creation du model avec transfert learning de vgg16 et ajout de couches de sortie
model = Sequential()

for layer in vgg16.layers:
    model.add(layer)

for layer in model.layers:
    layer.trainable= False

model.add(Flatten(input_shape= (4, 4, 512)))
model.add(Dropout(0.2))
model.add(Dense(4,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 37, 37, 256)       2

### Baseline Model Training

In [22]:
model.compile(
    optimizer= keras.optimizers.Adam(learning_rate= 0.0001), 
    loss='categorical_crossentropy', 
    metrics= ['accuracy']
    )

In [23]:
# définition des hyperparamètres du model
model_name='vgg16_e10b128'
checkpoint_filepath = model_name+'/tmp/checkpoint'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
earlystop = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1)
callbacks_list = [earlystop,checkpoint]

In [39]:
# entrainement 
numepochs = 5
batch_size = 128
history = model.fit(train_x , train_y,
                    epochs=numepochs, 
                    batch_size = batch_size
                    )


Epoch 1/5


TypeError: in user code:

    File "d:\02.Pro\SIMPLON\Certification\E2\e2_retinal_oct\.venv\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "d:\02.Pro\SIMPLON\Certification\E2\e2_retinal_oct\.venv\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\02.Pro\SIMPLON\Certification\E2\e2_retinal_oct\.venv\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "d:\02.Pro\SIMPLON\Certification\E2\e2_retinal_oct\.venv\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "d:\02.Pro\SIMPLON\Certification\E2\e2_retinal_oct\.venv\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "d:\02.Pro\S

### Evaluations on Test Dataset

In [ ]:
(eval_loss, eval_accuracy) = model.evaluate(test_x, test_y, batch_size= batch_size, verbose= 1)
print('Test Loss: ', eval_loss)
print('Test Accuracy: ', eval_accuracy)

In [ ]:
plt.subplot()
plt.rcParams['figure.figsize'] = (6.0, 4.0)
plt.title('Baseline Model Accuracy')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Training Accuracy','Validation Accuracy'])
plt.savefig(model_name+'/img/baseline_acc_epoch_'+model_name+'.png', transparent= False, bbox_inches= 'tight', dpi= 400)
plt.show()

In [ ]:

plt.subplot()
plt.title('Baseline Model Loss')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Training Loss','Validation Loss'])
plt.savefig(model_name+'/img/baseline_loss_epoch_'+model_name+'.png', transparent= False, bbox_inches= 'tight', dpi= 400)
plt.show()

In [ ]:
Y_pred = model.predict(test_generator, nb_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(test_generator.classes, y_pred)
df_cm = pd.DataFrame(cm, list(test_generator.class_indices.keys()), list(test_generator.class_indices.keys()))
fig, ax = plt.subplots(figsize=(10,8))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, cmap=plt.cm.Blues)
plt.title('Confusion Matrix\n')
plt.savefig(model_name+'/img/confusion_matrix_'+model_name+'.png', transparent= False, bbox_inches= 'tight', dpi= 400)
plt.show()


In [ ]:
print('Classification Report\n')
target_names = list(test_generator.class_indices.keys())
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

### Save the model

In [ ]:
model_save_h5 = model_name+"/retinal_oct_model_"+model_name+".h5"
model_save_json = model_name+"/retinal_oct_model_"+model_name+".json"
model_save_weights = model_name+"/retinal_oct_model_"+model_name+"_weights.h5"
model_save_metrics = model_name+"/retinal_oct_model_"+model_name+"_eval.json"

In [ ]:
# save model and architecture to h5 file
model.save(model_save_h5)
print("Saved h5 model to disk")

In [ ]:
#save the model architecture to JSON file
from keras.models import model_from_json
# serialize model to json
json_model = model.to_json()
with open(model_save_json, 'w') as json_file:
    json_file.write(json_model)
print("Saved json model to disk")

In [ ]:
#saving the weights of the model
model.save_weights(model_save_weights)
print("Saved model weights to disk")

In [ ]:
import json
# save model metrics
json_model_eval = {}
json_model_eval["model_name"]=model_name
json_model_eval["loss"] = eval_loss
json_model_eval["accuracy"] = eval_accuracy
with open(model_save_metrics, 'w') as json_file:
    json_file.write(str(json_model_eval))
print("Saved model metrics to disk")

### Load and evaluate models

In [ ]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model(model_save_h5)

# Let's check:
# np.testing.assert_allclose(
#     model.predict(test_generator), 
#     reconstructed_model.predict(test_generator)
# )

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
#reconstructed_model.fit(test_generator)


Evaluation du modèle vgg16 epochs 5 batchsize 128 nouvel entrainement :

In [ ]:
(eval_loss, eval_accuracy) = reconstructed_model.evaluate(test_generator, batch_size= batch_size, verbose= 1)
print('Test Loss: ', eval_loss)
print('Test Accuracy: ', eval_accuracy)

Evaluation du modèle vgg16 epochs 5 batchsize 128 model existant au début du projet :

In [ ]:
# récupération des métriques d'évaluation du modèle
e5b128_model_old = keras.models.load_model("../.old/model/retinal-oct.h5")
e5b128_model_old.evaluate(test_generator)

Evaluation du modèle vgg16 epochs 5 batchsize 128, modèle entrainé lors de la phase d'étude du projet :

In [ ]:
# récupération des métriques d'évaluation du modèle
e5b128_model = keras.models.load_model("vE5-B128/retinal_oct_model_vE5-B128.h5")
e5b128_model.evaluate(test_generator)

L'écart entre les valeurs des métriques de ces 3 modèles avec les même hyperparamètres peut s'expliquer avec un jeu de données différent.

### Prediction test

In [ ]:
from PIL import Image
import io
# model = tf.keras.models.load_model(model_save_h5)
reconstructed_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

test_image = keras.utils.load_img(test_data_dir+"/CNV/CNV-1016042-1.jpeg", target_size = (150, 150)) 
test_image = keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

#predict the result
result = np.argmax(model.predict(test_image))
print(result)
print(list(train_generator.class_indices.keys())[list(train_generator.class_indices.values()).index(result)])